In [ ]:
from hydra import compose, initialize
from lib.scene import Scene, GaussianModel
from lib.gaussian_renderer import Renderer
from lib.utils.mesh_utils import to_cam_open3d
from lib.utils.mesh_utils import GaussianExtractor
import matplotlib.pyplot as plt
import open3d as o3d
import numpy as np
import torch

def normal_to_world(camera, normals):
    # Assuming extrinsic is a 4x4 matrix and you're using the top-left 3x3 rotation matrix
    extrinsic_matrix = torch.tensor(camera.extrinsic[:3, :3]).to(torch.float32)
    # Reshape normals to shape (3, 100*100) to apply the transformation
    normals_flat = normals.view(3, -1)
    # Multiply the rotation matrix (3x3) with each normal vector (3, N)
    transformed_normals = torch.matmul(extrinsic_matrix, normals_flat)
    # Reshape back to (3, 100, 100)
    transformed_normals = transformed_normals.view(3, *normals.shape[1:])
    return transformed_normals

with initialize(config_path="./conf", version_base=None):
    cfg = compose(config_name="train", overrides=[])

cfg.dataset.model_path = "/home/borth/2d-gaussian-splatting/logs/2025-02-13/17-10-07"
render = Renderer(pipe=cfg.pipeline, dataset=cfg.dataset)
gaussians = GaussianModel(cfg.dataset.sh_degree)
scene = Scene(cfg.dataset, gaussians, load_iteration=-1)

extractor = GaussianExtractor(scene.gaussians, render)
sh_degree = extractor.gaussians.active_sh_degree
extractor.gaussians.active_sh_degree = 0
extractor.reconstruction(viewpoint_stack=scene.getTrainCameras())
extractor.gaussians.active_sh_degree = sh_degree

In [ ]:
img = extractor.rgbmaps[0].permute(1, 2, 0)
plt.imshow(img.detach().cpu().numpy())

mesh = extractor.extract_mesh_bounded()
o3d.visualization.draw_plotly([mesh])

In [ ]:
N = 500_000

cameras = to_cam_open3d(extractor.viewpoint_stack)
np_points = []
np_normals = []
for idx, camera in enumerate(cameras):
    # create from depth map the world coordinates
    depth_image = extractor.depthmaps[idx].detach().cpu().numpy()
    depth = o3d.geometry.Image(depth_image[0])
    intrinsic = camera.intrinsic
    extrinsic = camera.extrinsic
    pcd = o3d.geometry.PointCloud.create_from_depth_image(depth, intrinsic, extrinsic)
    np_points.append(np.asarray(pcd.points))
    # compute the normals
    normal = extractor.normals[idx].permute(1, 2, 0).reshape(-1, 3)
    # normal = normal_to_world(camera, extractor.normals[idx]).permute(1, 2, 0).reshape(-1, 3)
    np_normals.append(normal.detach().cpu().numpy())


np_points = np.vstack(np_points)
np_normals = np.vstack(np_normals)

indices = np.random.choice(np_points.shape[0], size=N, replace=False)
points = o3d.utility.Vector3dVector(np_points[indices])
normals = o3d.utility.Vector3dVector(np_normals[indices])
pcd = o3d.geometry.PointCloud(points)
# pcd.normals = normals
pcd.estimate_normals()
pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=0.5)

In [ ]:
o3d.visualization.draw_plotly([pcd])

In [ ]:
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9)
o3d.io.write_triangle_mesh("/home/borth/2d-gaussian-splatting/tmp/poisson9_mesh105_estimate.ply", mesh)
o3d.visualization.draw_plotly([mesh])

In [ ]:
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9)
# Compute a threshold based on density percentiles
density_threshold = np.percentile(np.asarray(densities), 5)  # Keep top 95% dense areas
# Select vertices above density threshold
vertices_to_keep = np.asarray(densities) > density_threshold
mesh.remove_vertices_by_mask(~vertices_to_keep)
# Visualize the cleaned mesh
o3d.visualization.draw_plotly([mesh])